# Dates and Timestamps

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('dates').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/13 17:31:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/13 17:31:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/10/13 17:31:55 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/10/13 17:31:55 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
df = spark.read.csv('appl_stock.csv',inferSchema=True, header=True)

In [4]:
df.head(1)

[Row(Date=datetime.datetime(2010, 1, 4, 0, 0), Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039)]

In [5]:
df.select(['Date', 'Open']).show()

+-------------------+------------------+
|               Date|              Open|
+-------------------+------------------+
|2010-01-04 00:00:00|        213.429998|
|2010-01-05 00:00:00|        214.599998|
|2010-01-06 00:00:00|        214.379993|
|2010-01-07 00:00:00|            211.75|
|2010-01-08 00:00:00|        210.299994|
|2010-01-11 00:00:00|212.79999700000002|
|2010-01-12 00:00:00|209.18999499999998|
|2010-01-13 00:00:00|        207.870005|
|2010-01-14 00:00:00|210.11000299999998|
|2010-01-15 00:00:00|210.92999500000002|
|2010-01-19 00:00:00|        208.330002|
|2010-01-20 00:00:00|        214.910006|
|2010-01-21 00:00:00|        212.079994|
|2010-01-22 00:00:00|206.78000600000001|
|2010-01-25 00:00:00|202.51000200000001|
|2010-01-26 00:00:00|205.95000100000001|
|2010-01-27 00:00:00|        206.849995|
|2010-01-28 00:00:00|        204.930004|
|2010-01-29 00:00:00|        201.079996|
|2010-02-01 00:00:00|192.36999699999998|
+-------------------+------------------+
only showing top

In [6]:
from pyspark.sql.functions import (dayofmonth,hour,
                                  dayofyear,month,
                                  year,weekofyear,
                                  format_number,date_format)

In [13]:
# day of the month
df.select(['Date', dayofmonth(df['Date'])]).show(3)

+-------------------+----------------+
|               Date|dayofmonth(Date)|
+-------------------+----------------+
|2010-01-04 00:00:00|               4|
|2010-01-05 00:00:00|               5|
|2010-01-06 00:00:00|               6|
+-------------------+----------------+
only showing top 3 rows



In [14]:
# hour
df.select(['Date', hour(df['Date'])]).show(3)

+-------------------+----------+
|               Date|hour(Date)|
+-------------------+----------+
|2010-01-04 00:00:00|         0|
|2010-01-05 00:00:00|         0|
|2010-01-06 00:00:00|         0|
+-------------------+----------+
only showing top 3 rows



In [15]:
# month
df.select(['Date', month(df['Date'])]).show(3)

+-------------------+-----------+
|               Date|month(Date)|
+-------------------+-----------+
|2010-01-04 00:00:00|          1|
|2010-01-05 00:00:00|          1|
|2010-01-06 00:00:00|          1|
+-------------------+-----------+
only showing top 3 rows



In [16]:
# year
df.select(['Date', year(df['Date'])]).show(3)

+-------------------+----------+
|               Date|year(Date)|
+-------------------+----------+
|2010-01-04 00:00:00|      2010|
|2010-01-05 00:00:00|      2010|
|2010-01-06 00:00:00|      2010|
+-------------------+----------+
only showing top 3 rows



In [17]:
# average closing price per year
df.select(year(df['Date'])).show(3)

+----------+
|year(Date)|
+----------+
|      2010|
|      2010|
|      2010|
+----------+
only showing top 3 rows



In [20]:
# add column year to the dataframe
df.withColumn('Year', year(df['Date'])).show(1)

+-------------------+----------+----------+------------------+----------+---------+---------+----+
|               Date|      Open|      High|               Low|     Close|   Volume|Adj Close|Year|
+-------------------+----------+----------+------------------+----------+---------+---------+----+
|2010-01-04 00:00:00|213.429998|214.499996|212.38000099999996|214.009998|123432400|27.727039|2010|
+-------------------+----------+----------+------------------+----------+---------+---------+----+
only showing top 1 row



In [23]:
new_df = df.withColumn('Year', year(df['Date']))

In [35]:
result = new_df.groupBy('Year').mean().select(
    'Year', 
    'avg(Close)'
)

In [42]:
#result.withColumnRenamed('avg(Close)', 'Average Closing Price').show()
result.select([
    'Year', 
    format_number('avg(Close)',2).alias('Average Closing Price')
]).show()

+----+---------------------+
|Year|Average Closing Price|
+----+---------------------+
|2015|               120.04|
|2013|               472.63|
|2014|               295.40|
|2012|               576.05|
|2016|               104.60|
|2010|               259.84|
|2011|               364.00|
+----+---------------------+

